In [ ]:
from keras.applications import Xception # TensorFlow ONLY
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import numpy as np
import argparse
import cv2
import re
import os
import matplotlib.pyplot as plt
import pandas as pd
from keras import optimizers

import warnings
warnings.filterwarnings('ignore')

from keras.utils import np_utils
from keras.models import Model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D,Concatenate, Flatten, Dense, Dropout, BatchNormalization, LeakyReLU

In [ ]:
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
lemmatizer=WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
from IPython import display


# Adjust the output height to 300px.
display.Javascript("google.colab.output.setIframeHeight('100px');")

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!unzip /content/drive/My\ Drive/Resized_Images_299.zip #done

Archive:  /content/drive/My Drive/Resized_Images_299.zip
   creating: Resized_Images_299/
  inflating: Resized_Images_299/jim_when-people-use-memes-where-they-think-jim-carrey-says-55582907.png  
  inflating: Resized_Images_299/mrbri_126falls-asleep-in-class-bad-luck-brian-memes-before-wet-32669985.png  
  inflating: Resized_Images_299/best_2018_571a0c73c694b2b5daeba50f6eb1d263_large.jpg  
  inflating: Resized_Images_299/titanic_chinese-titanic-NfoIE.jpg  
  inflating: Resized_Images_299/deadpool_13081l.jpg  
  inflating: Resized_Images_299/obama_13217-3.jpg  
  inflating: Resized_Images_299/feminist_cg6t801.jpg  
  inflating: Resized_Images_299/zeck_l-1995-mark-zuckerberg-dropping-off-his-senior-executives-at-the-uk-parliament.jpg  
  inflating: Resized_Images_299/cat_U_389dog-and-cat-memes-48.jpg  
  inflating: Resized_Images_299/obama_762.jpg  
  inflating: Resized_Images_299/bean_593e7b5a93640.jpeg  
  inflating: Resized_Images_299/trump_trump-imbecile-genius.jpg  
  inflating: Res

In [ ]:
! wget http://nlp.stanford.edu/data/glove.6B.zip

--2019-11-25 17:10:05--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-11-25 17:10:06--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2019-11-25 17:10:06--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
! unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
def preprocess(X):
  corpus = []
  for review in X:
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

  return corpus

FOR **POSITIVE**


In [ ]:
num = re.compile(r'(\d+)')
def key_func(a):
    ids = num.split(a)
    ids[1::2] = map(int, ids[1::2])
    return ids

data_df = pd.read_csv('/content/drive/My Drive/updated_data.csv')
y_train = data_df.Offensive.tolist()

In [ ]:
train = "/content/Resized_Images_299/"
#train = "/content/drive/My Drive/NNFL Fall'19 Project/data_7000/"
x_train_image=[]
i=1
for file in data_df['Image']:
  try:
    img = train + file
    image = cv2.resize(cv2.imread(img), (100, 100))
    x_train_image.append(image)
    len(x_train_image)

  except Exception as e:
    print(str(e))



In [ ]:
x_train_image = np.asarray(x_train_image)
x_train_image = x_train_image.astype('float32')
x_train_image /= 255.0

#y_train = pd.get_dummies(y_train)  #preprocessing the image

In [ ]:
x_train_image.shape

(6507, 100, 100, 3)

In [ ]:
data_df.head()

Image  ... Positive
0                                 10_year_2r94rv.jpg  ...        4
1          10_year_10-year-challenge_1547788782.jpeg  ...        4
2  10_year_10yearchallenge-5c75f8b946e0fb0001edc7...  ...        3
3  10_year_10-year-challenge-sweet-dee-edition-40...  ...        3
4  10_year_10-year-challenge-with-no-filter-47-hi...  ...        2

[5 rows x 7 columns]

In [ ]:
x_train_text = data_df['Caption']
x_train_text = preprocess(x_train_text)
x_train_text

['look friend lightyear sohalikut trend play year challenge facebook imgflip com',
 'best yearchallenge completed le year kudu narendramodi ji pm jan mumbai india',
 'sam thorne strippin follow follow saw everyone posting v pic mine pm jan retweets like k',
 'year challenge sweet dee edition',
 'year challenge filter hilarious year challenge meme year challenge',
 'get car stranger meet people internet alone memecenter com uber order stranger internet get car alone',
 'year challenge lit af b erg wow transformation',
 'year challenge emotional edition boredpanda com',
 'fornite died month minecraft never died year proud community',
 'facebook year challenge ploy simple meme mo exo h minh le jan',
 'probably first man year challenge chennai pasanga da',
 'state dining room white house v boredpanda com',
 'facebook year challenge kind went year future instead back instant regret',
 'ifidownloada movie jamaica meme ami pirate caribbean pick food floor sec bacteria meme angry fuck',
 'anti

In [ ]:
from numpy import asarray

embeddings_index = dict()
f = open('glove.6B.300d.txt', encoding='utf8')
for line in f:
	values = line.split()
	word = ''.join(values[:-300])
	coefs = asarray(values[-300:], dtype='float32')
	embeddings_index[word] = coefs
f.close()

In [ ]:
MAX_NB_WORDS = 3000
MAX_SEQUENCE_LENGTH = 105
EMBEDDING_DIM = 300

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(x_train_text)
word_index = tokenizer.word_index

vocab_size = len(word_index)+1

# encoded_docs = tokenizer.texts_to_sequences(X_train)
# print(encoded_docs)
# # pad documents to a max length of 4 words
# max_length = 4
# padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
# print(padded_docs)

In [ ]:
x_train_text = tokenizer.texts_to_sequences(x_train_text)
x_train_text = pad_sequences(x_train_text, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of desc tensor:', x_train_text.shape)

Y = pd.get_dummies(y_train).values
print('Shape of rating tensor:', Y.shape)

Shape of desc tensor: (6507, 105)
Shape of rating tensor: (6507, 4)


In [ ]:
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
from sklearn.model_selection import train_test_split
#x_train_text, x_test_text, x_train_text, Y_test_text = train_test_split(X_text,Y, test_size = 0.10, random_state = 42)
#print(x_train.shape,y_train.shape)
#print(x_test.shape,y_test.shape)

In [ ]:
print(x_train_image.shape)
print(x_train_text.shape)

(6507, 100, 100, 3)
(6507, 105)


In [ ]:

model_x = Xception(
  include_top=False,
  weights='imagenet', 
  #input_tensor=None, 
  input_shape=(100,100,3),
  pooling=None
)

model_x.trainable=False
model_x.summary()













83689472/83683744 [==============================] - 3s 0us/step
Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100, 100, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 49, 49, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 49, 49, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 49, 49, 32)   0           block1_conv1_bn[0][0]            
______________

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Input, SpatialDropout1D, Dropout
from keras.callbacks import ModelCheckpoint

input_text = Input(shape = (105,))
input_image = Input(shape = (100,100,3))

model_image = Sequential()
model_image.add(model_x)
model_image.add(Flatten())
model_image.add(Dense(8,activation = 'relu'))
model_image.add(Dense(4,activation = 'sigmoid'))
model_image.summary()
#model_image = Model(input_shape, model_temp)

#inputA = Input(shape=(32,))
#inputB = Input(shape=(128,))



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 3, 3, 2048)        20861480  
_________________________________________________________________
flatten_1 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 147464    
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 36        
Total params: 21,008,980
Trainable params: 147,500
Non-trainable params: 20,861,480
_________________________________________________________________


In [ ]:
#OPTIONAL - TO SEE HOW THE IMAGE MODEL PERFORMS ON ITS OWN
model_image.compile(loss='binary_crossentropy',optimizer = optimizers.Adam(lr=5e-5), metrics = ['accuracy'])

model_filename_image = 'offensive_image.h5'
checkpoint_image = ModelCheckpoint(model_filename_image, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='max', period=1)

callbacks_list_image = [checkpoint_image]

model_image.fit(x_train_image,Y, epochs = 30,validation_split=0.1, batch_size = 32, callbacks=callbacks_list_image)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 5856 samples, validate on 651 samples
Epoch 1/30
5856/5856 [==============================] - 11s 2ms/step - loss: 0.5729 - acc: 0.7307 - val_loss: 0.5264 - val_acc: 0.7304

Epoch 00001: val_acc improved from -inf to 0.73041, saving model to offensive_image.h5
Epoch 2/30
5856/5856 [==============================] - 5s 787us/step - loss: 0.5474 - acc: 0.7437 - val_loss: 0.5293 - val_acc: 0.7224

Epoch 00002: val_acc did not improve from 0.73041
Epoch 3/30
5856/5856 [==============================] - 5s 776us/step - loss: 0.5378 - acc: 0.7436 - val_loss: 0.5271 - val_acc: 0.7323

Epoch 00003: val_acc improved from 0.73041 to 0.73233, saving model to offensive_image.h5
Epoch 4/30
5856/5856 [==============================] - 5s 785us/step - loss: 0.5301 - acc: 0.7498 - val_loss: 0.5295 - val_acc: 0.7193

Epoch 00004: val_acc did not improve from 0.73233
Epoch 5/30
5856/5856 [=========

In [ ]:
model_text = Sequential()
model_text.add(Embedding(vocab_size, EMBEDDING_DIM, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, trainable=False))
model_text.add(SpatialDropout1D(0.2))
model_text.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model_text.add(Dense(4,activation = 'sigmoid'))
model_text.summary()




#model_x.trainable=True

#for layer in model_x.layers[:-3]:
#    layer.trainable = False


#model_image.add(Dense(32, activation='relu'))
#model_image.add(Dense(1, activation='sigmoid'))

#x = Dense(4, activation="relu")(combinedInput)
#x = Dense(1, activation="linear")(x)

#merged = Concatenate([model_image, model_text])




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 105, 300)          3259200   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 105, 300)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                93440     
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 260       
Total params: 3,352,900
Trainable params: 93,700
Non-trainable params: 3,259,200
_________________________________________________________________


In [ ]:
#OPTIONAL - TO SEE HOW THE TEXT MODEL PERFORMS ON ITS OWN

model_text.compile(loss='binary_crossentropy',optimizer = optimizers.Adam(lr=1e-4), metrics = ['accuracy'])

model_filename_text = 'offensive_text.h5'
checkpoint_text = ModelCheckpoint(model_filename_text, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='max', period=1)

callbacks_list_text = [checkpoint_text]

model_text.fit(x_train_text,Y, epochs = 30,validation_split=0.1, batch_size = 64, callbacks=callbacks_list_text)

Train on 5856 samples, validate on 651 samples
Epoch 1/30
5856/5856 [==============================] - 17s 3ms/step - loss: 0.6473 - acc: 0.6332 - val_loss: 0.5721 - val_acc: 0.7323

Epoch 00001: val_acc improved from -inf to 0.73233, saving model to offensive_text.h5
Epoch 2/30
5856/5856 [==============================] - 16s 3ms/step - loss: 0.5429 - acc: 0.7419 - val_loss: 0.5156 - val_acc: 0.7515

Epoch 00002: val_acc improved from 0.73233 to 0.75154, saving model to offensive_text.h5
Epoch 3/30
5856/5856 [==============================] - 16s 3ms/step - loss: 0.5024 - acc: 0.7490 - val_loss: 0.5016 - val_acc: 0.7500

Epoch 00003: val_acc did not improve from 0.75154
Epoch 4/30
5856/5856 [==============================] - 15s 3ms/step - loss: 0.4971 - acc: 0.7499 - val_loss: 0.5014 - val_acc: 0.7500

Epoch 00004: val_acc did not improve from 0.75154
Epoch 5/30
5856/5856 [==============================] - 15s 3ms/step - loss: 0.4961 - acc: 0.7500 - val_loss: 0.5017 - val_acc: 0.7500

In [ ]:
from keras.models import load_model

#model_image = load_model('offensive_image.h5')
#model_text = load_model('offensive_text.h5')

combinedInput = Concatenate(axis=-1)([model_image.output, model_text.output])
result = Dense(8, activation="relu")(combinedInput)
result = Dense(4, activation="sigmoid")(result)

result_model = Model(inputs=[model_image.input, model_text.input], outputs=result)

In [ ]:
result_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
xception_input (InputLayer)     (None, 100, 100, 3)  0                                            
__________________________________________________________________________________________________
embedding_1_input (InputLayer)  (None, 105)          0                                            
__________________________________________________________________________________________________
xception (Model)                (None, 3, 3, 2048)   20861480    xception_input[0][0]             
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 105, 300)     3259200     embedding_1_input[0][0]          
____________________________________________________________________________________________

In [ ]:
adam = optimizers.Adam(lr=1e-4)

result_model.compile(loss='binary_crossentropy',optimizer = adam, 
    metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
model_filename = 'offensive.h5'
checkpoint = ModelCheckpoint(model_filename, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='max', period=1)

callbacks_list = [checkpoint]
result_model.fit([x_train_image,x_train_text], Y, validation_split=0.1, validation_data=None, shuffle=True, epochs=30, batch_size=32, callbacks=callbacks_list)#, class_weight = {0: 0.4, 1: 0.6})

5856/5856 [==============================] - 57s 10ms/step - loss: 0.4846 - acc: 0.7500 - val_loss: 0.5042 - val_acc: 0.7500

Epoch 00025: val_acc did not improve from 0.75000
Epoch 26/30

Epoch 00025: val_acc did not improve from 0.75000
Epoch 26/30
5856/5856 [==============================] - 57s 10ms/step - loss: 0.4833 - acc: 0.7500 - val_loss: 0.5042 - val_acc: 0.7500

Epoch 00026: val_acc did not improve from 0.75000
Epoch 27/30

Epoch 00026: val_acc did not improve from 0.75000
Epoch 27/30
5856/5856 [==============================] - 57s 10ms/step - loss: 0.4818 - acc: 0.7500 - val_loss: 0.5043 - val_acc: 0.7500

Epoch 00027: val_acc did not improve from 0.75000
Epoch 28/30

Epoch 00027: val_acc did not improve from 0.75000
Epoch 28/30
5856/5856 [==============================] - 56s 10ms/step - loss: 0.4802 - acc: 0.7500 - val_loss: 0.5043 - val_acc: 0.7500

Epoch 00028: val_acc did not improve from 0.75000
Epoch 29/30

Epoch 00028: val_acc did not improve from 0.75000
Epoch 29

In [ ]:
from google.colab import files

files.download('offensive.h5')